## Load model

In [ ]:
import torch
device = torch.device('cpu')
# layer segmentation
import os
import time
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import os.path as osp
from os.path import exists
import argparse
import json
import logging
import time
import copy
##### pytorch library #####
import torch
from torch import nn
import torch.backends.cudnn as cudnn
from torch.autograd import Variable

##### My own library #####
import data.seg_transforms as dt
from data.seg_dataset import segList
from utils.logger import Logger
from models.net_builder import net_builder
from utils.loss import loss_builder1,loss_builder2,ch_loss_builder2,ch_loss_builder3
from utils.utils import adjust_learning_rate
from utils.utils import AverageMeter,save_model
from utils.utils import compute_dice,compute_pa,compute_single_avg_score
from utils.vis import vis_result
import numpy as np
# import segmentation_models_pytorch as smp

In [ ]:
pt_file = torch.load('./output/checkpoint_00600.pt')
model = net_builder('unet')
# model = smp.Unet(encoder_name="resnet50",in_channels=1,classes=8)
pt_file.keys()
model.load_state_dict(pt_file['model_state'])
model.to(device)
info = json.load(open(osp.join('/scratch/kedu04/Segmentation/data/dataset_public', 'info.json'), 'r'))
normalize = dt.Normalize(mean=info['mean'], std=info['std'])
t = []
t.extend([dt.Label_Transform(),dt.ToTensor(),normalize])
train_dataset = segList('/scratch/kedu04/Segmentation/data/dataset_public', 'train', dt.Compose(t))

In [ ]:
#train_dataset, _ = torch.utils.data.random_split(train_dataset, [30,783])

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=1, shuffle=True, num_workers=2, pin_memory=True, drop_last=True)

In [ ]:
criterion2 = ch_loss_builder3()
criterion2 = criterion2.to(device)

## Train Loss Graph

In [ ]:
train_loss = []
epoch=600
for i in range(1,epoch):
    pt_file = torch.load(f'./output/checkpoint_{str(i).zfill(5)}.pt')
    train_loss.append(pt_file['val_loss']['dice'])

In [ ]:
import matplotlib.pyplot as plt
plt.title("Train loss")
plt.plot(np.arange(1,epoch,1),train_loss)
# plt.plot(valid_loss)
plt.ylabel("Generalized Dice Coeffecient Loss")
plt.xlabel("epoch")
plt.show()

## Visualization

In [ ]:
import matplotlib.pyplot as plt
model.eval()
count = 0
for iter, (input, target) in enumerate(train_loader):
    
    with torch.no_grad():

        tick_iter=time.time()
        # variable
        input_var = Variable(input).to(device)

        target_var_seg = Variable(target).to(device)
        # forward
        output_seg = model(input_var)
        loss_DICELOSS = criterion2(output_seg, target_var_seg)
        loss = loss_DICELOSS     # loss from the two-stage network   
        tock_iter = time.time()
        duration_iter = tock_iter-tick_iter


        str_iter="iter: {}/{} Dice:{} Total:{} duration:{}ms".format(iter,len(train_loader),loss_DICELOSS.item(),loss.item(),duration_iter)
        #print(str_iter)
        input = input.squeeze(0)
        target = target.squeeze(0)
        output_seg = output_seg.squeeze(0)
        #input = input.permute(1,2,0)

        #output_seg = output_seg.permute(1,2,0)
        output_seg = np.array(output_seg)
        output_seg = np.argmax(output_seg, axis = 0)
        input = np.array(input)
        input = input[0]
        target = np.array(target)
        print(iter, "target:",np.unique(target,return_counts=True))
        print(iter, "output:",np.unique(output_seg,return_counts=True))
        fig, (ax1, ax2, ax3) = plt.subplots(1, 3,figsize=(10, 5))
        ax1.set_title("input")
        ax1.imshow(input)
        ax1.axis('off')

        ax2.set_title(f"predicted (Dice: {round(1.-loss_DICELOSS.item(),2)})")
        ax2.imshow(output_seg)
        ax2.axis('off')

        ax3.set_title("target")
        ax3.imshow(target)
        ax3.axis('off')
        plt.show()
        real_seg = output_seg
        real_input = input
        real_target= target